## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>


<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# Import libraries
from pyspark.sql.types import *
from pyspark.sql.functions import input_file_name, col, current_timestamp, sum, desc, size, split, udf, count, when, to_timestamp, explode, regexp_extract, regexp_replace, ltrim, substring
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import time
import datetime

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# Read the source file directory listing
source_file_listing = get_source_listing_df()

In [0]:
# Count the source files
file_number = source_file_listing['File Name'].count()
print(f"Number of source files: {file_number}")

In [0]:
# Print the contents of one of the files
print("Content example of a file:")
print(show_s3_file_contents('voc_volume/1.json'))

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Define the schema for the raw data
raw_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True)
])

In [0]:
# Setup a read stream using cloudfiles and the source data format
raw_stream = (spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .schema(raw_schema)
    .load(TWEET_SOURCE_PATH)
    .withColumn('source_file',input_file_name())
    .withColumn('processing_time',current_timestamp())
)

In [0]:
# Define the schema for the bronze data
bronze_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField('source_file', StringType(), True),
    StructField('processing_time', TimestampType(), False)
])

In [0]:
# Setup a write stream using cloudfiles to append to the bronze delta table
bronze_stream = (
    raw_stream
    .writeStream
    .option("mergeSchema","true")
    .outputMode("append")
    .format('delta')
    .queryName("bronze_stream")
    .option("checkpointLocation",BRONZE_CHECKPOINT)
    .start(BRONZE_DELTA)
)

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# Load data from the Bronze Delta table
bronze_table = spark.read.format("delta").load(BRONZE_DELTA)

In [0]:
# Number of tweets captured in the Bronze Table
tweet_count = bronze_table.count()
print(f"Number of tweets captured in the Bronze Table: {tweet_count}")

In [0]:
# Display columns and the number of Nan or Null values:
print("Display columns and the number of Nan or Null values:")
columns_with_null = bronze_table.select([count(when(col(c).isNull(), c)).alias(c) for c in bronze_table.columns]).show()
display(columns_with_null)


In [0]:
# Count the number of tweets by each unique user handle and sort the data by descending count.
user_tweet_counts = bronze_table.groupBy("user").count().orderBy(col('count').desc())
display(user_tweet_counts)

In [0]:
# Number of tweets having at least one mention (@)
tweet_mention = bronze_table.filter(col("text").like("%@%")).count()
print(f"Number of tweets having at least one mention (@): {tweet_mention}")

In [0]:
# Number of tweets having no mention (@):
tweet_no_mention = bronze_table.filter(~ col("text").like("%@%")).count()
print(f"Number of tweets having no mention (@): {tweet_no_mention}")

In [0]:
# Top 20 tweeters by number counts
top_20_users = user_tweet_counts.limit(20).toPandas()

# Plot a bar chart that shows the top 20 tweeters (users)
plt.bar(top_20_users['user'], top_20_users['count'],color='skyblue')
plt.xticks(rotation=45, ha="right")
plt.xlabel('User')
plt.ylabel('Number of Tweets')
plt.title('Top 20 Users with Most Tweet Count')
plt.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Setup a read stream on your bronze delta table
bronze_read_stream = (
    spark
    .readStream
    .format('delta')
    .load(BRONZE_DELTA)
)

In [0]:
# Prepare data for silver stream
silver_data = (
    bronze_read_stream
    .withColumn("timestamp", to_timestamp(col("date"), "EEE MMM dd HH:mm:ss zzz yyyy"))

    .withColumn("mention", explode(split(col("text"), " ")))
    .filter(col("mention").startswith("@"))

    .withColumn("cleaned_text",regexp_replace(col('text'),"(\s|^)@\w+",""))

    .select("timestamp","mention","cleaned_text","sentiment")
)

In [0]:
# Setup a write stream to append to the silver delta table
silver_stream = (
    silver_data
    .writeStream
    .option("mergeSchema","true")
    .outputMode("append")
    .format('delta')
    .queryName("silver_stream")
    .option("checkpointLocation",SILVER_CHECKPOINT)
    .start(SILVER_DELTA)
)

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# Setup a read stream on your silver delta table
silver_read_stream = (spark
                .readStream
                .format('delta')
                .load(SILVER_DELTA))

In [0]:
# Create a Spark UDF for the production model
model_uri = f"models:/{MODEL_NAME}/production"
model_udf = mlflow.pyfunc.spark_udf(spark, model_uri)

In [0]:
# Prepare data for gold stream
gold_data_df = (
    silver_read_stream
    .withColumn("predicted",model_udf(col("cleaned_text")))
    .withColumn("predicted_score", col("predicted.score").cast("double"))
    .withColumn("predicted_sentiment", col("predicted.label"))
    .withColumn("sentiment_id",
                when(col('sentiment') == 'positive',1)
                .otherwise(0))
    .withColumn("predicted_sentiment_id",
                when(col('predicted_sentiment') == "POS",1)
                .when(col('predicted_sentiment') == "NEG",0)
                .otherwise(-1))
    .select('timestamp', 'mention', 'cleaned_text', 'sentiment', 'predicted_score', 'predicted_sentiment','sentiment_id', 'predicted_sentiment_id')
)

In [0]:
# Setup a write stream to append to the silver delta table
gold_stream = (gold_data_df
               .writeStream
               .format("delta")
               .outputMode("append")
               .queryName('gold_stream')
               .option("checkpointLocation", GOLD_CHECKPOINT)
               .start(GOLD_DELTA)
               )

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# Load the gold table
gold_table = (spark.read.format('delta').load(GOLD_DELTA))

In [0]:
# Extract the output and remove the predicted segment as Neutral. I decided to remove neutral prediction out of performance evaluation, because the actual sentiment only has 2 values: positive and negative. And it seems not reasonable to treat neutral tweets as negative ones, leading to unnecessary consequences when applying treatments which are used for negative tweets to them.
model_output_refined = (
    gold_table
    .filter(col('predicted_sentiment') != 'NEU')
    .select('predicted_sentiment', "sentiment_id","predicted_sentiment_id")
    .toPandas()
)

In [0]:
# Get the actual and predicted labels
y_true = model_output_refined['sentiment_id'].astype(int)
y_pred = model_output_refined['predicted_sentiment_id'].astype(int)

# Calculate precision, recall and f1 score
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(11, 7))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

In [0]:
with mlflow.start_run():
    # Store the precision, recall, and F1-score as MLflow metrics
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)

    # Store an image of the confusion matrix as an MLflow artifact
    mlflow.log_artifact("confusion_matrix.png", "confusion_matrix.png")

    # Store the model name and the MLflow version
    mlflow.log_param("model name", MODEL_NAME)
    mlflow.log_param("mlflow version", mlflow.version.VERSION)

    # Store the version of the Delta Table (input-silver) as an MLflow parameter
    # Extract the delta version
    delta_table = DeltaTable.forPath(spark, SILVER_DELTA)
    delta_version = delta_table.history().orderBy("version").select("version").collect()[-1][0]
    # Log the delta version
    mlflow.log_param("delta_version", delta_version)

    mlflow.end_run()


## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# Extract the full output (including predicted sentiment as Neutral)
model_output = (
    gold_table
    .toPandas()
)

In [0]:
# Number of unique mentions in the gold data
unique_mention = model_output['mention'].nunique()
print(f"Number of unique mentions in the gold data: {unique_mention}")


In [0]:
# Count the number of negative, positive, and neutral tweets for each mention. 
# Capture the total for each mention in a new column
# Sort the mention count totals in descending order
mention_agg = (gold_table.groupBy("mention").agg(
     sum(when(col("Predicted_sentiment") == "POS", 1).otherwise(0)).alias("positive_count"),
     sum(when(col("Predicted_sentiment") == "NEU", 1).otherwise(0)).alias("neutral_count"),
     sum(when(col("Predicted_sentiment") == "NEG", 1).otherwise(0)).alias("negative_count"))
     .withColumn('total_mentions', col('positive_count') + col('neutral_count') + col('negative_count'))
     .select('mention', 'total_mentions', 'positive_count', 'neutral_count', 'negative_count')
     .orderBy(col('total_mentions').desc()))

# Display mention aggregation
display(mention_agg)

In [0]:
# Top 20 positive mentions
top_20_pos_mention = mention_agg.sort(col('positive_count').desc()).select('mention', 'positive_count').limit(20).toPandas().set_index('mention')

# Plot a bar chart of the top 20 mentions with positive sentiment
plt.figure(figsize=(10, 6))
sns.barplot(x=top_20_pos_mention.index, y='positive_count', data=top_20_pos_mention, color='skyblue')
plt.xlabel('Mentions')
plt.ylabel('Positive Count')
plt.title('Top 20 Mentions with Positive Sentiment Count')
plt.xticks(rotation=90)
plt.show()

In [0]:
# Top 20 negative mentions
top_20_neg_mention = mention_agg.sort(col('negative_count').desc()).select('mention', 'negative_count').limit(20).toPandas().set_index('mention')

# Plot a bar chart of the top 20 mentions with negative sentiment
plt.figure(figsize=(10, 6))
sns.barplot(x=top_20_neg_mention.index, y='negative_count', data=top_20_neg_mention, color='skyblue')
plt.xlabel('Mentions')
plt.ylabel('Negative Count')
plt.title('Top 20 Mentions with Negative Sentiment Count')
plt.xticks(rotation=90)
plt.show()

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# Print the active stream
for stream in spark.streams.active:
    print(f"Stream Name: {stream.name} is running")

In [0]:
# Stop all streams
stop_all_streams()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()
TOTAL_TIME = END_TIME - START_TIME 

# Convert timestamp to hour:minute:second
start_time_formatted = datetime.datetime.fromtimestamp(START_TIME).strftime('%H:%M:%S')
end_time_formatted = datetime.datetime.fromtimestamp(END_TIME).strftime('%H:%M:%S')
total_time_rounded = round(TOTAL_TIME, 2)

# Print times and duration
print(f"Start Time: {start_time_formatted}")
print(f"End Time: {end_time_formatted}")
print(f"Total Running Duration: {total_time_rounded} s")

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### Spark Optimization
1. Spill
- During the analysis, I couldn't find any spill information, suggesting sufficient memory allocation to executors and optimal performance.
(https://drive.google.com/file/d/1vrSoMzbFtz7JI2tFn-KDFCxJpO0zZmSS/view)


2. Skew
- I do not notice any significant imblalance data in partition size. However, we can enhance the pipeline efficiency by predefining the number of clusters, especially considering the wide transformations like groupBy, aggregate, and distinct that it involves.

3. Shuffle
- There are many Shuffle Read and Shuffle Write activities during the whole run, indicating that aggregation operations were causing data shuffling across the cluster. Optimizing transformations to reduce shuffle to improve overall performance by minimizing disk and network I/O overhead may help increase the performance. (https://drive.google.com/file/d/1vrSoMzbFtz7JI2tFn-KDFCxJpO0zZmSS/view).


4. Storage
- I noticed that the storage format appeared inefficient due to a large number of small JSON files. To improve code efficiency, we must address this issue by optimizing the handling of these numerous small files.

5. Serialization
- There is no serialization except for the model_udf.

